In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)
from sklearn.utils import shuffle

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import *
import tensorflow_addons as tfa
tf.__version__

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

# Hyperparameters

In [ ]:
BEST_PATH = './models/ResNet.h5'
TRAINING_EPOCHS = 200
LEARNING_RATE = 0.002
EPSILON = 1e-06
BATCH_SIZE = 16

# Data loading

In [ ]:
l = np.load('./results/2020_S/fw_dataset.npz', allow_pickle=True)
data_indices = l['data_indices']
input_data = l['input_data']
output_label = l['output_label']
INPUT_MAXS = l['INPUT_MAXS']
INPUT_MINS = l['INPUT_MINS']
OUTPUT_MAX = l['OUTPUT_MAX']
OUTPUT_MIN = l['OUTPUT_MIN']

In [ ]:
input_data = input_data.astype('float32')
output_label = output_label.astype('float32')

In [ ]:
print(input_data.shape)
print(output_label.shape)

In [ ]:
print(INPUT_MAXS)
print(INPUT_MINS)

In [ ]:
print(OUTPUT_MAX)
print(OUTPUT_MIN)

In [ ]:
data_indices, input_data, output_label = shuffle(data_indices, input_data, output_label, random_state=3101)

In [ ]:
N_TRAIN = int(input_data.shape[0]*.8)
train_input = input_data[:N_TRAIN, ...]
train_label = output_label[:N_TRAIN, ...]
val_input = input_data[N_TRAIN:, ...]
val_label = output_label[N_TRAIN:, ...]

In [ ]:
print(f'number of training set: {train_input.shape[0]}')
print(f'number of validation set: {val_input.shape[0]}')

In [ ]:
with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=False)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_input, val_label))
    val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=False)

# Model construction

In [ ]:
class ResNet1D(Model):
    def __init__(self):
        super(ResNet1D, self).__init__()
        self.n = [128, 128, 256, 256, 512] # number of nodes
        self.k = [1, 5, 10, 20, 50] # kernal size
        self.s = 2 # stride (= pooling size)
                
        self.conv1_1 = layers.Conv1D(self.n[0]/4, self.k[0], kernel_initializer='glorot_normal', padding='same')
        self.conv1_2 = layers.Conv1D(self.n[0]/4, self.k[1], kernel_initializer='glorot_normal', padding='same')
        self.conv1_3 = layers.Conv1D(self.n[0]/4, self.k[2], kernel_initializer='glorot_normal', padding='same')
        self.conv1_4 = layers.Conv1D(self.n[0]/4, self.k[3], kernel_initializer='glorot_normal', padding='same')
        self.conv1_5 = layers.Conv1D(self.n[0]/4, self.k[4], kernel_initializer='glorot_normal', padding='same')
        self.batch1 = layers.BatchNormalization()
        self.activation1 = layers.Activation(tf.nn.leaky_relu)
        self.pool1 = layers.MaxPooling1D(2)
        
        self.conv2 = layers.Conv1D(self.n[1], 1, kernel_initializer='glorot_normal', padding='valid')
        self.batch2 = layers.BatchNormalization()
        self.activation2 = layers.Activation(tf.nn.leaky_relu)
        self.pool2 = layers.MaxPooling1D(2)
        
        self.conv3 = layers.Conv1D(self.n[2], 1, kernel_initializer='glorot_normal', padding='valid')
        self.batch3 = layers.BatchNormalization()
        self.activation3 = layers.Activation(tf.nn.leaky_relu)
        self.pool3 = layers.MaxPooling1D(2)
                                          
        self.conv4 = layers.Conv1D(self.n[3], 1, kernel_initializer='glorot_normal', padding='valid')
        self.batch4 = layers.BatchNormalization()
        self.activation4 = layers.Activation(tf.nn.leaky_relu)
        self.pool4 = layers.MaxPooling1D(2)
        
        self.output_conv = layers.Conv1D(self.n[4], 1, kernel_initializer='glorot_normal', padding='valid')
        self.gate = layers.Dense(1, activation=tf.nn.sigmoid)
        
    def call(self, inp):
        
        inp = tf.concat([self.conv1_1(inp), self.conv1_2(inp), self.conv1_3(inp), self.conv1_4(inp), self.conv1_5(inp)], -1)
        inp = self.pool1(self.activation1(self.batch1(inp)))
        
        inp = self.pool2(self.activation2(self.batch2(self.conv2(inp))))
        inp = self.pool3(self.activation3(self.batch3(self.conv3(inp))))
        inp = self.pool4(self.activation4(self.batch4(self.conv4(inp))))
        
        inp = self.gate(self.output_conv(inp))
        
        return inp

In [ ]:
callbacks = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=.5, patience=2, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0)

save = tf.keras.callbacks.ModelCheckpoint(
    BEST_PATH, monitor='val_loss', verbose=0,
    save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20) 

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    model = ResNet1D()
    model.compile(optimizer=opt, loss='mae')
    model.fit(train_dataset, epochs=TRAINING_EPOCHS, validation_data=val_dataset,
                  verbose=1, callbacks=[callbacks, save, early_stop]) 

In [ ]:
model.load_weights(BEST_PATH)

In [ ]:
model.evaluate(val_dataset)